In [1]:
import numpy as np
import pandas as pd
import pprint
import json

In [2]:
data_csv = pd.read_csv('../data/all-nodes.csv')

In [24]:
def all_children(id):
    '''For a given node id, it return all the children of that node'''
    while id != None:
#         print(id)
        children = eval(list(data_csv.children[data_csv.id == id])[0])
        if len(children) > 0:
            id = children[0]
        else:
            break
            
all_children(1)

In [4]:
def find_all_ancestors(id):
    
    child = []
    while id != None:
        cat_name = data_csv.name[data_csv.id == id].values[0]
        parent_id = data_csv.parent[data_csv.id == id].values[0]
        productCount = data_csv.subtreeProductCount[data_csv.id == id].values[0]
        
        ancestors = {}
        ancestors['id'] = int(id)
        ancestors['name'] = cat_name
        ancestors['value'] = int(productCount)
        ancestors['children'] = child
        child = [ancestors]
#         print(id,cat_name)
        id = parent_id
        if id == 0:
#             print(id,'root')
            ancestors = {}
            ancestors['id'] = 0
            ancestors['name'] = 'root'
            ancestors['value'] = int(0)
            ancestors['children'] = child
            return ancestors
            break

In [5]:
def find_all_hierarchy(cat_name):
    all_hierarchy = []
    all_cat_ids = data_csv.id.loc[data_csv.name == cat_name].values
    for cat_id in all_cat_ids:
        hierarchy = find_all_ancestors(cat_id)
        all_hierarchy.append(hierarchy)
#         print("#####"*20)
    return all_hierarchy
#     return json.dumps(all_hierarchy)

In [20]:
find_all_hierarchy('Boots')

[{'id': 0,
  'name': 'root',
  'value': 0,
  'children': [{'id': 4,
    'name': 'Clothing, Shoes & Jewelry',
    'value': 1503384,
    'children': [{'id': 237,
      'name': 'Men',
      'value': 371334,
      'children': [{'id': 1417,
        'name': 'Shoes',
        'value': 95218,
        'children': [{'id': 7519,
          'name': 'Boots',
          'value': 13991,
          'children': []}]}]}]}]},
 {'id': 0,
  'name': 'root',
  'value': 0,
  'children': [{'id': 4,
    'name': 'Clothing, Shoes & Jewelry',
    'value': 1503384,
    'children': [{'id': 237,
      'name': 'Men',
      'value': 371334,
      'children': [{'id': 7479,
        'name': 'Surf, Skate & Street',
        'value': 27727,
        'children': [{'id': 7480,
          'name': 'Shoes',
          'value': 9158,
          'children': [{'id': 7520,
            'name': 'Boots',
            'value': 1287,
            'children': []}]}]}]}]}]},
 {'id': 0,
  'name': 'root',
  'value': 0,
  'children': [{'id': 4,
    'nam

In [7]:
def merge_trees(main_tree, side_tree):
    '''Given two trees (a main tree and a side tree having only max one child) ,
    it merges them and returns the merged tree'''
    #pprint.pprint(main_tree)
    #pprint.pprint(side_tree)
    if len(main_tree) == 0:
        return side_tree
    main_children = main_tree['children']
    side_child = side_tree['children'][0]
    if side_child == None:
        return main_tree
    if main_children[0] == None:
        return side_tree
    children_ids = [c['id'] for c in main_children]
    if side_child != None and side_child['id'] in children_ids:
#         print(side_child['id'],children_ids)
#         print(np.argwhere(np.array(children_ids) == side_child['id'])[0][0])
#         print('--'*10)
        idx = np.argwhere(np.array(children_ids) == side_child['id'])[0][0]
        child_merged =  merge_trees(main_tree['children'][idx],side_child)
        main_tree['children'][idx] = child_merged
        return main_tree
    else:
        main_tree['children'].append(side_tree['children'][0])
        #pprint.pprint(main_tree)
        return main_tree

In [8]:
def merge_all_trees(all_trees):
    merged_tree = {}
    for tree in all_trees:
        merged_tree = merge_trees(merged_tree,tree)
        
    return merged_tree

In [21]:
all_trees = find_all_hierarchy('Boots')

In [22]:
merged_tree = merge_all_trees(all_trees)
merged_tree

{'id': 0,
 'name': 'root',
 'value': 0,
 'children': [{'id': 4,
   'name': 'Clothing, Shoes & Jewelry',
   'value': 1503384,
   'children': [{'id': 237,
     'name': 'Men',
     'value': 371334,
     'children': [{'id': 1417,
       'name': 'Shoes',
       'value': 95218,
       'children': [{'id': 7519,
         'name': 'Boots',
         'value': 13991,
         'children': []}]},
      {'id': 7479,
       'name': 'Surf, Skate & Street',
       'value': 27727,
       'children': [{'id': 7480,
         'name': 'Shoes',
         'value': 9158,
         'children': [{'id': 7520,
           'name': 'Boots',
           'value': 1287,
           'children': []}]}]}]},
    {'id': 1538,
     'name': 'Boys',
     'value': 60338,
     'children': [{'id': 8193,
       'name': 'Shoes',
       'value': 16831,
       'children': [{'id': 8194,
         'name': 'Boots',
         'value': 3119,
         'children': []}]}]},
    {'id': 5,
     'name': 'Girls',
     'value': 78291,
     'children': [{'i

In [23]:
json.dumps(merged_tree)

'{"id": 0, "name": "root", "value": 0, "children": [{"id": 4, "name": "Clothing, Shoes & Jewelry", "value": 1503384, "children": [{"id": 237, "name": "Men", "value": 371334, "children": [{"id": 1417, "name": "Shoes", "value": 95218, "children": [{"id": 7519, "name": "Boots", "value": 13991, "children": []}]}, {"id": 7479, "name": "Surf, Skate & Street", "value": 27727, "children": [{"id": 7480, "name": "Shoes", "value": 9158, "children": [{"id": 7520, "name": "Boots", "value": 1287, "children": []}]}]}]}, {"id": 1538, "name": "Boys", "value": 60338, "children": [{"id": 8193, "name": "Shoes", "value": 16831, "children": [{"id": 8194, "name": "Boots", "value": 3119, "children": []}]}]}, {"id": 5, "name": "Girls", "value": 78291, "children": [{"id": 5066, "name": "Shoes", "value": 21518, "children": [{"id": 8326, "name": "Boots", "value": 4510, "children": []}]}]}, {"id": 231, "name": "Women", "value": 837929, "children": [{"id": 1648, "name": "Shoes", "value": 245248, "children": [{"id":

In [25]:
np.unique(["#6cbe55", "#e0819f", "#5ac4f8", "#f5603a", "#b98829", "#e76cef", "#8e80fb", "#0cc0aa", "#fb4e93", "#a08d7f", "#fa1bfc", "#00d618"]+
         ["#6cbe55", "#cc9cc7", "#35afa6", "#d759f3", "#ba865c", "#6895bc", "#ef6268", "#f2ac18", "#8e80fb", "#fb7810", "#00d618"]+
         ["#0cc0aa", "#d783b9", "#02b72e", "#dc58ea", "#e2712e", "#6ec1f8", "#769d31", "#e9af2a", "#a88c65", "#ff4d82", "#7487fb"]+
         ["#8cb3ab", "#d9acc2", "#0db293", "#ff4d82", "#d85ee1", "#f37e2f", "#abc34d", "#59d13e", "#ca9519", "#ac8bf8", "#4c93e9", "#fe16f4"])

array(['#00d618', '#02b72e', '#0cc0aa', '#0db293', '#35afa6', '#4c93e9',
       '#59d13e', '#5ac4f8', '#6895bc', '#6cbe55', '#6ec1f8', '#7487fb',
       '#769d31', '#8cb3ab', '#8e80fb', '#a08d7f', '#a88c65', '#abc34d',
       '#ac8bf8', '#b98829', '#ba865c', '#ca9519', '#cc9cc7', '#d759f3',
       '#d783b9', '#d85ee1', '#d9acc2', '#dc58ea', '#e0819f', '#e2712e',
       '#e76cef', '#e9af2a', '#ef6268', '#f2ac18', '#f37e2f', '#f5603a',
       '#fa1bfc', '#fb4e93', '#fb7810', '#fe16f4', '#ff4d82'], dtype='<U7')

In [35]:
def get_all_children(catId):
    parent = {}
    children = []
    children_ids = eval(data_csv.children.loc[data_csv.id == catId].values[0])
    parent_id = int(catId);
    parent_name = data_csv.name[data_csv.id == catId].values[0]
    parent_sub_roduct_count = data_csv.subtreeProductCount[data_csv.id == catId].values[0]
    parent_roduct_count = data_csv.productCount[data_csv.id == catId].values[0]
    
    for child_id in children_ids:
        cat_name = data_csv.name[data_csv.id == child_id].values[0]
        sub_roduct_count = data_csv.subtreeProductCount[data_csv.id == child_id].values[0]
        roduct_count = data_csv.productCount[data_csv.id == child_id].values[0]
        
        child = {}
        child['id'] = int(child_id)
        child['name'] = cat_name
        child['value'] = int(sub_roduct_count)
        child['roduct_count'] = int(roduct_count)
        children.append(child)
    parent['id'] = parent_id
    parent['name'] = parent_name
    parent['value'] = int(parent_sub_roduct_count)
    parent['product_count'] = int(parent_roduct_count)
    parent['children'] = children
    return parent

In [36]:
json.dumps(get_all_children(2060))

'{"id": 2060, "name": "Snow Sports", "value": 11966, "product_count": 12, "children": [{"id": 2061, "name": "Skiing", "value": 8355, "roduct_count": 361}, {"id": 5961, "name": "Sledding", "value": 417, "roduct_count": 17}, {"id": 8025, "name": "Snowboarding", "value": 2399, "roduct_count": 150}, {"id": 9425, "name": "Snowmobiling Equipment", "value": 252, "roduct_count": 178}, {"id": 9441, "name": "Snowshoeing", "value": 543, "roduct_count": 14}]}'